In [1]:
import numpy as np
import pandas as pd
import PIL
import glob
import random
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
from ultralytics import YOLO
import cv2
from IPython.display import Image, display

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = YOLO('/workspace/jay/DDP/Ocelot/yolo_binary/runs/detect/train/weights/best.pt')
files = sorted(glob.glob('/workspace/jay/DDP/Ocelot/ocelot2023/images/train/cell/*.jpg'))
#temp = glob.glob('/workspace/jay/DDP/Ocelot/yolo_binary/datasets/cell_detect_33-1/test/images/*.jpg')

In [3]:
val = glob.glob('/workspace/jay/DDP/Ocelot/yolo_binary/datasets/cell_detect_33-1/valid/images/*.jpg')
test = glob.glob('/workspace/jay/DDP/Ocelot/yolo_binary/datasets/cell_detect_33-1/test/images/*.jpg')
val_files = np.unique(np.array([x.split('/')[-1][:3] for x in val]))
test_files = np.unique(np.array([x.split('/')[-1][:3] for x in test]))

In [4]:
for k in tqdm(range(40)):
    color=(255,0,0)
    file = val_files[k]
    #file = test_files[k]
    idx = int(file) -1 
    path = f'/workspace/jay/DDP/Ocelot/ocelot2023/images/train/cell/{file}.jpg'
    annot_path = f'/workspace/jay/DDP/Ocelot/ocelot2023/annotations/train/cell/{file}.csv'
    df = pd.read_csv(annot_path,header=None)
    img1 = np.array(PIL.Image.open(path))
    img2 = copy.deepcopy(img1)

    out = model.predict(path,conf=0.2,iou=0.5)
    out = out[0].cpu().numpy()
    boxes = out.boxes.data

    for i in range(len(df)):
        x,y = df.loc[i][0],df.loc[i][1]
        clas = df.loc[i][2] - 1
        start = (x-17,y-17)
        end = (x+17,y+17)
        if clas==0:
            cv2.rectangle(img1, start, end, (0,255,0),2)     # class = 0, BC, Green
        else:
            cv2.rectangle(img1, start, end, (255,0,0),2)     # class = 1, TC, Red

    for i in range(len(boxes)):
        box = boxes[i][:4].astype(int)
        x,y = min(1023,int((boxes[i][0]+boxes[i][2])/2)), min(1023,int((boxes[i][1]+boxes[i][3])/2))
        #prob = boxes[i][4]
        start = (x-17,y-17)
        end = (x+17,y+17)
        clas = boxes[i][5] 
        if clas==0:
            cv2.rectangle(img2, start, end, (0,255,0),2)    # class = 0, BC, Green
        else:
            cv2.rectangle(img2, start, end, (255,0,0),2)    # class = 1, TC, Red

    final_img = np.concatenate([img1,np.zeros((1024,50,3)).astype(int)],axis=1)
    finall_img = np.concatenate([final_img,img2],axis=1)

    plt.ioff()
    f = plt.figure()
    f.set_figwidth(18)
    f.set_figheight(9)
    plt.imshow(finall_img)
    plt.title('Val: '+file)
    f.savefig(f'/workspace/jay/DDP/Ocelot/yolo_binary/visualize/val/{file}.png')
    plt.close(f)
    #plt.imshow(finall_img)        
                        
            
                                ###Ground truth                                                    ### Predicted

  0%|                                                                                                                                                                               | 0/40 [00:00<?, ?it/s]Ultralytics YOLOv8.0.20 🚀 Python-3.8.13 torch-1.13.1+cu117 CUDA:0 (NVIDIA A100-SXM4-80GB, 81251MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:23<00:00,  1.67it/s]
